In [1]:
import logging

from kazoo.client import KazooClient
from kazoo.recipe.barrier import DoubleBarrier
from threading import Thread
from time import sleep

logging.basicConfig()

In [2]:
NAMESPACE = '/zoo'

N_ANIMALS = 2

In [3]:
zk = KazooClient()
zk.start()

if zk.exists(NAMESPACE):
    zk.delete(NAMESPACE, recursive=True)
    
zk.create(NAMESPACE)

'/zoo'

In [4]:
# @zk.ChildrenWatch(NAMESPACE)
# def watch_children(children):
#     print('Current children list:', children)
#     if len(children) < self.party_size:
#         print('Waiting for the others.')
#     elif len(children) == self.party_size:
#         print('Zoo is full')
#     else:
#         print('Zoo is crowded')


In [5]:
class Animal:
    
    def __init__(self, name, root, party_size):
        self.root = root
        self.name = name
        self.url = f'{root}/{name}'
        self.party_size = party_size
        
        self.zk = KazooClient()
        self.zk.start()
        
        self.zkb = DoubleBarrier(self.zk, f'{self.root}_barrier', self.party_size)
        
    def enter(self)    :
        self.zk.create(self.url, ephemeral=True)
                
        @self.zk.ChildrenWatch(self.root)
        def watch_node(children):
            # just reacting on the callback, let's just print zoo content
            if len(children) < self.party_size:
                print(f'{self.name} callback | "Zoo is capable to accept."\n') # Waiting for the others.')
            elif len(children) == self.party_size:
                print(f'{self.name} callback | "Zoo is full."\n')
            else:
                print(f'{self.name} callback | "Zoo is crowded."\n')
        
        print(f'{self.name} is waiting....\n')
        self.zkb.enter()
        # only when all animals are in there, barrier would be released
        print(f'{self.name} RUNS!\n')
        self.zkb.leave()
                      
    
    def leave(self):
        self.zk.delete(self.url)
        self.zk.stop()
        


In [6]:
class AnimalThread(Thread):
    
    def __init__(self, name, root, party_size):
        super().__init__()
        self.root = root
        self.name = name
        self.party_size = party_size
        
    def run(self):
        animal = Animal(self.name, self.root, self.party_size)
        animal.enter()
        animal.leave()

In [7]:
animals = ['cat', 'dog', 'elephant']

In [8]:
# for name in animals:
#     AnimalThread(name, NAMESPACE, len(animals)).start()

In [9]:
AnimalThread(animals[0], NAMESPACE, len(animals)).start()

cat callback | "Zoo is capable to accept."

cat is waiting....



In [10]:
AnimalThread(animals[1], NAMESPACE, len(animals)).start()

dog callback | "Zoo is capable to accept."
cat callback | "Zoo is capable to accept."

dog is waiting....




In [11]:
AnimalThread(animals[2], NAMESPACE, len(animals)).start()

dog callback | "Zoo is full."
elephant callback | "Zoo is full."

elephant is waiting....

cat callback | "Zoo is full."


dog RUNS!
cat RUNS!


elephant RUNS!

cat callback | "Zoo is capable to accept."

elephant callback | "Zoo is capable to accept."

dog callback | "Zoo is capable to accept."

cat callback | "Zoo is capable to accept."

dog callback | "Zoo is capable to accept."



## MUST wait for all the threads before cleanup

In [12]:
# clearing all trash in the end
zk.delete(NAMESPACE, recursive=True)
zk.delete(f'{NAMESPACE}_barrier', recursive=True)

In [13]:
zk.get_children("/")

['zookeeper']

In [14]:
zk.stop()
zk.close()